## Setup

In [3]:
!pip install gurobipy numpy pandas seaborn matplotlib

  Obtaining dependency information for seaborn from https://files.pythonhosted.org/packages/7b/e5/83fcd7e9db036c179e0352bfcd20f81d728197a16f883e7b90307a88e65e/seaborn-0.13.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/294.6 kB ? eta -:--:--
   ------------------------------------- -- 276.5/294.6 kB 8.6 MB/s eta 0:00:01
   -------------------------------------- - 286.7/294.6 kB 5.9 MB/s eta 0:00:01
   ---------------------------------------- 294.6/294.6 kB 3.1 MB/s eta 0:00:00


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import numpy as np
import gurobipy as gp
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

## Loading the 2019 and 2020 file and initializing variables

In [5]:
#loading the 2019 file
df_2019 = pd.read_csv(r'./stocks2019.csv')
df_2020 = pd.read_csv(r'./stocks2020.csv')

#setting the time limit
time = 3600

if len(df_2019.columns) - 2 <= 5:
    print('Please have more stocks in the data before proceeding ahead!!')

#calculating the number of stocks in the data
num_stocks = len(df_2019.columns) - 2

# creating array for m values
m_values = [5,10,20,30,40,50,60,70,80,90,100]
m = [x for x in m_values if x <= num_stocks]

#renaming the first column to date
df_2019.rename(columns = {list(df_2019)[0] : 'Date'}, inplace = True)
df_2020.rename(columns = {list(df_2020)[0] : 'Date'}, inplace = True)

# Convert the 'Date' column into a datetime object
df_2019['Date'] = pd.to_datetime(df_2019['Date'])
df_2020['Date'] = pd.to_datetime(df_2020['Date'])

# Set 'Date' as the index
df_2019.set_index('Date', inplace=True)
df_2020.set_index('Date', inplace=True)

C:\Users\nikna\AppData\Local\Temp\ipykernel_21476\2604927873.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_2020['Date'] = pd.to_datetime(df_2020['Date'])


### Calculating the stock return similarity *matrix*

In [7]:
#creating a dataframe containing daily returns of stocks
df_2019_returns = df_2019/df_2019.shift(1) - 1

#calculating correlation between different stocks and creating the matrix
df_corr = df_2019_returns.corr().iloc[1:,1:]

In [8]:
df_corr

,ATVI,ADBE,AMD,ALXN,ALGN,GOOGL,GOOG,AMZN,AMGN,ADI,...,TCOM,ULTA,VRSN,VRSK,VRTX,WBA,WDAY,WDC,XEL,XLNX
ATVI,1.000000,0.399939,0.365376,0.223162,0.216280,0.433097,0.426777,0.467076,0.203956,0.329355,...,0.322906,0.128241,0.464850,0.316549,0.259679,0.218149,0.311659,0.303077,0.043389,0.249667
ADBE,0.399939,1.000000,0.452848,0.368928,0.363370,0.552125,0.540404,0.598237,0.291978,0.473815,...,0.360392,0.201151,0.711339,0.541243,0.402171,0.228106,0.650430,0.361516,0.207403,0.289497
AMD,0.365376,0.452848,1.000000,0.301831,0.344252,0.418861,0.417254,0.549302,0.151452,0.503733,...,0.332776,0.210623,0.498342,0.330900,0.272983,0.281950,0.407626,0.438892,0.017283,0.478010
ALXN,0.223162,0.368928,0.301831,1.000000,0.332433,0.315993,0.307698,0.363170,0.342022,0.317040,...,0.257143,0.408936,0.350581,0.191489,0.522423,0.192720,0.416396,0.289908,0.047947,0.200356
ALGN,0.216280,0.363370,0.344252,0.332433,1.000000,0.248747,0.250316,0.399281,0.264599,0.328280,...,0.175957,0.128559,0.360886,0.251855,0.334978,0.219595,0.308968,0.284407,0.088059,0.253934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WBA,0.218149,0.228106,0.281950,0.192720,0.219595,0.232900,0.230603,0.288168,0.194490,0.347861,...,0.257049,0.145330,0.195475,0.126118,0.214585,1.000000,0.211641,0.296309,-0.130798,0.276643
WDAY,0.311659,0.650430,0.407626,0.416396,0.308968,0.379493,0.371826,0.424748,0.211712,0.351734,...,0.235045,0.269545,0.569672,0.401476,0.376047,0.211641,1.000000,0.315699,0.209240,0.254117
WDC,0.303077,0.361516,0.438892,0.289908,0.284407,0.328619,0.322110,0.419620,0.172623,0.602935,...,0.377215,0.126463,0.331916,0.308558,0.314773,0.296309,0.315699,1.000000,-0.076115,0.516958
XEL,0.043389,0.207403,0.017283,0.047947,0.088059,0.059930,0.052570,0.076724,0.137857,-0.047259,...,-0.172752,0.074686,0.280371,0.407770,0.191667,-0.130798,0.209240,-0.076115,1.000000,-0.117552


## 1. Stock Allocation

### Stock Selection

In [19]:
m = 5

# Gurobi time limit
gurobi_time_limit = 3600

index_name = df_2019.columns.values[:1][0]
print(f'1st Stock: {index_name}\n')

stock_names = df_2019.columns.values[1:]
print(f'Stock Names: {stock_names}\n')

num_stocks = len(stock_names)

1st Stock: NDX

Stock Names: ['ATVI' 'ADBE' 'AMD' 'ALXN' 'ALGN' 'GOOGL' 'GOOG' 'AMZN' 'AMGN' 'ADI'
 'ANSS' 'AAPL' 'AMAT' 'ASML' 'ADSK' 'ADP' 'BIDU' 'BIIB' 'BMRN' 'BKNG'
 'AVGO' 'CDNS' 'CDW' 'CERN' 'CHTR' 'CHKP' 'CTAS' 'CSCO' 'CTXS' 'CTSH'
 'CMCSA' 'CPRT' 'COST' 'CSX' 'DXCM' 'DOCU' 'DLTR' 'EBAY' 'EA' 'EXC' 'EXPE'
 'FB' 'FAST' 'FISV' 'GILD' 'IDXX' 'ILMN' 'INCY' 'INTC' 'INTU' 'ISRG' 'JD'
 'KLAC' 'KHC' 'LRCX' 'LBTYA' 'LBTYK' 'LULU' 'MAR' 'MXIM' 'MELI' 'MCHP'
 'MU' 'MSFT' 'MRNA' 'MDLZ' 'MNST' 'NTES' 'NFLX' 'NVDA' 'NXPI' 'ORLY'
 'PCAR' 'PAYX' 'PYPL' 'PEP' 'PDD' 'QCOM' 'REGN' 'ROST' 'SGEN' 'SIRI'
 'SWKS' 'SPLK' 'SBUX' 'SNPS' 'TMUS' 'TTWO' 'TSLA' 'TXN' 'TCOM' 'ULTA'
 'VRSN' 'VRSK' 'VRTX' 'WBA' 'WDAY' 'WDC' 'XEL' 'XLNX']



In [10]:
mod = gp.Model()

y = mod.addMVar(num_stocks, vtype="B")
x = mod.addMVar((num_stocks, num_stocks), vtype="B")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19


In [24]:
# number of stocks in fund constraint
size_fund_c = mod.addConstr( gp.quicksum(y[i] for i in range(num_stocks)) == m)

# # all stocks represented by single stock constraint
stock_incl_c = mod.addConstrs( gp.quicksum( x[i][j] for j in range(num_stocks) ) == 1 for i in range(num_stocks))

# each stock represented by a stock on the fund
stock_rep_c = mod.addConstrs( x[i][j] <= y[j] for i in range(num_stocks) for j in range(num_stocks))

# Set objective value
mod.setObjective( gp.quicksum( x[i][j]*df_corr.iloc[i,j] for i in range(num_stocks) for j in range(num_stocks)) , gp.GRB.MAXIMIZE)

In [25]:
mod.Params.output_flag = 0
mod.optimize()

In [26]:
mod.ObjVal

54.83990652229107

In [27]:
selection_array = y.X
selected_stocks = stock_names[ y.X == 1]
selected_stocks

array(['LBTYK', 'MXIM', 'MSFT', 'VRTX', 'XEL'], dtype=object)

### Stock Weights

In [28]:
num_t = len(df_2019_returns)

In [31]:
mod_w = gp.Model()
w = mod_w.addMVar(m)
y = mod_w.addMVar(num_t)

# Add of weigths of selected stocks equal to 1
w_cons = mod_w.addConstr( gp.quicksum(w[i] for i in range(m)) == 1)
# Absolute value constraints
abs_cons1 = mod_w.addConstrs( y[t] >= df_2019_returns[index_name][t] - gp.quicksum(w[i]*df_2019_returns[selected_stocks[i]][t] for i in range(m) ) for t in range(num_t))
abs_cons2 = mod_w.addConstrs( y[t] >= gp.quicksum(w[i]*df_2019_returns[selected_stocks[i]][t] for i in range(m) ) - df_2019_returns[index_name][t] for t in range(num_t))

mod_w.setObjective( gp.quicksum(y[t] for t in range(num_t)) )
mod_w.Params.output_flag = 0
mod_w.optimize()

C:\Users\nikna\AppData\Local\Temp\ipykernel_21476\4253313308.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  abs_cons1 = mod_w.addConstrs( y[t] >= df_2019_returns[index_name][t] - gp.quicksum(w[i]*df_2019_returns[selected_stocks[i]][t] for i in range(m) ) for t in range(num_t))


GurobiError: Element 0 of a double array is Nan.

In [30]:
stock_weights = w.X
stock_weights

GurobiError: Index 0 out of range for attribute 'X'